# Inception score 
code from: https://machinelearningmastery.com/how-to-implement-the-inception-score-from-scratch-for-evaluating-generated-images/

In [ ]:

# calculate inception score for cifar-10 in Keras
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from numpy.random import shuffle
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets import cifar10
from skimage.transform import resize
from numpy import asarray
 
# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)
 
# assumes images have any shape and pixels in [0,255]
def calculate_inception_score(images, n_split=10, eps=1E-16):
	# load inception v3 model
	model = InceptionV3()
	# enumerate splits of images/predictions
	scores = list()
	n_part = floor(images.shape[0] / n_split)
	for i in range(n_split):
		# retrieve images
		ix_start, ix_end = i * n_part, (i+1) * n_part
		subset = images[ix_start:ix_end]
		# convert from uint8 to float32
		subset = subset.astype('float32')
		# scale images to the required size
		subset = scale_images(subset, (299,299,3))
		# pre-process images, scale to [-1,1]
		subset = preprocess_input(subset)
		# predict p(y|x)
		p_yx = model.predict(subset)
		# calculate p(y)
		p_y = expand_dims(p_yx.mean(axis=0), 0)
		# calculate KL divergence using log probabilities
		kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
		# sum over classes
		sum_kl_d = kl_d.sum(axis=1)
		# average over images
		avg_kl_d = mean(sum_kl_d)
		# undo the log
		is_score = exp(avg_kl_d)
		# store
		scores.append(is_score)
	# average across images
	is_avg, is_std = mean(scores), std(scores)
	return is_avg, is_std


# IS in Cifar 10 from the tutorial

In [3]:
# load cifar10 images
(images, _), (_, _) = cifar10.load_data()
# shuffle images
shuffle(images)
print('loaded', images.shape)
# calculate inception score
is_avg, is_std = calculate_inception_score(images[:10000])
print('score', is_avg, is_std)

loaded (50000, 32, 32, 3)
32/32 [==============================] - 31s 965ms/step
score 11.081058 0.20513362


## IS in EM images

In [9]:
import os
import numpy
from PIL import Image
# list the images in augmented data
perc_train = 0.8
N = 1000

dirpath = os.getcwd()
data_path = os.path.join(dirpath, "augmented_data") # read some of the aumented data
img_list = numpy.array([filename.split(".jpg")[0] for filename in os.listdir(data_path)])
# split the data grouping by original image
group_list = numpy.array(["_".join(filename.split("_")[:2]) for filename in img_list])
group_list_un = numpy.unique(group_list)
num_train = numpy.round(len(group_list_un)*perc_train).astype('int')
numpy.random.seed(seed=2)
train_groups = numpy.random.choice(group_list_un, size=num_train, replace=False)
train_idx =sum([list(numpy.where(group_list == val)[0]) for val in train_groups], [])
test_idx = list(set(range(len(group_list))).difference(set(train_idx)))

train_im_list = img_list[train_idx]
test_im_list = img_list[test_idx]

train_imgs = numpy.array([numpy.asarray(Image.open(os.path.join(data_path, filename + ".jpg"))) for filename in train_im_list])

In [10]:
shuffle(train_imgs)
print('loaded', train_imgs.shape)
# calculate inception score
is_avg, is_std = calculate_inception_score(train_imgs)
print('score', is_avg, is_std)

loaded (7680, 256, 256, 3)
24/24 [==============================] - 25s 1s/step
score 3.3171794 0.0733383
